In [ ]:
%%time
import requests, psycopg2, random, os
from dotenv import load_dotenv

DB_ENV_PROD=0
load_dotenv()
api_key=os.getenv('API_KEY')

url = f'https://fmpcloud.io/api/v3/symbol/available-forex-currency-pairs?apikey={api_key}'
res = requests.get(url).json()
symbols = [response['symbol'] for response in res]

columns = [f'{symbol} float(24)' for symbol in symbols]
columns.insert(0, "date TIMESTAMP NOT NULL PRIMARY KEY")
columns = str(tuple(columns)).replace("'", '').replace('-', '_').replace('=', '_')

if DB_ENV_PROD==0:
    database=os.getenv('LOCAL_DB_NAME')
    user=os.getenv('LOCAL_DB_USER')
    password=os.getenv('LOCAL_DB_PASSWORD')
    port=int(os.getenv('DB_PORT'))
    host=os.getenv('DB_HOST')

    def database_connection(host=host,database=database,user=user,password=password,port=port):
        conn = psycopg2.connect(host=host, database=database, user=user, password=password, port=port)
        return conn

    conn = database_connection()
    cur = conn.cursor()

elif DB_ENV_PROD==1:    
    REMOTE_HOST = os.getenv('REMOTE_HOST')
    REMOTE_USERNAME = os.getenv('REMOTE_USERNAME')
    PKEY_PATH= './humble.pem'

    conn_params = {
        'database': os.getenv('CLOUD_DB_NAME'), 
        'user': os.getenv('CLOUD_DB_USER'), 
        'password': os.getenv('CLOUD_DB_PASSWORD'), 
        'host': os.getenv('CLOUD_DB_HOST'), 
        'port': int(os.getenv('CLOUD_DB_PORT'))
    }
    
    def database_connection():
        conn = psycopg2.connect(**conn_params)
        return conn
    
    conn = database_connection()
    cur = conn.cursor()
    

# TABLE CREATION
command=f'''
        CREATE TABLE IF NOT EXISTS forex_close_prices {columns};
        CREATE TABLE IF NOT EXISTS forex_volume_prices {columns};
        '''
try:
    cur.execute(command)
    conn.commit()
except:
    conn.close()
    conn = database_connection()
    cur = conn.cursor()
    cur.execute(command)
    conn.commit()

    
# INITIAL CHECKING FOR ALL SYMBOLS IN DB, ADD IF NOT
def unique_columns(x):
    if x.lower().strip().replace('=', '_') in symbols_in_DB:
        return False
    else:
        return True

types = ['close', 'volume']

for each_type in types:
    
    symbols_in_DB = []

    query = f'''
    SELECT * FROM information_schema.columns WHERE table_name = 'forex_{each_type}_prices';
    '''
    try:
        cur.execute(query)
    except:
        conn.close()
        conn=database_connection()
        cur = conn.cursor()
        cur.execute(query)
    lst=cur.fetchall()
    symbols_in_DB += [i[3] for i in lst]

    columns_not_in_DB = list(filter(unique_columns, symbols))
    
    if len(columns_not_in_DB) != 0:
        q=[f'ADD COLUMN {i} float(24)' for i in columns_not_in_DB]
        q[0] = f'ALTER TABLE forex_{each_type}_prices {q[0]}'
        alter_query = str(q).replace('[', '').replace("'", '').replace(']', ';')
        try:
            cur.execute(alter_query)
            conn.commit()
        except:
            conn.close()
            conn=database_connection()
            cur = conn.cursor()
            cur.execute(alter_query)
            conn.commit()
            

for symbol in symbols:
    url = f'https://fmpcloud.io/api/v3/historical-price-full/{symbol}?timeseries=1000&apikey={api_key}'
    res = requests.get(url).json()
    
    for each_day in res['historical']:
        close_data, volume_data = [], []
        
        close_data = [each_day["date"], each_day["close"]]
        
        try:
            volume_data = [each_day["date"], each_day["volume"]]
        except (KeyError):
            pass
        
        insert_query = f'''
                INSERT INTO forex_close_prices {str(('date', symbol.lower().replace("=", '_'))).replace("'", '')}
                VALUES (%s, %s)
                ON CONFLICT (date) DO UPDATE SET
                {symbol.lower().replace("=", '_')} = EXCLUDED.{symbol.lower().replace("=", '_')};
            '''
        
        try:
            cur.execute(insert_query, tuple(close_data))
            conn.commit()
        except:
            conn.close()
            conn=database_connection()
            cur = conn.cursor()
            cur.execute(insert_query, tuple(close_data))
            conn.commit()
        
        if volume_data:
            insert_query = f'''                
                INSERT INTO forex_volume_prices {str(('date', symbol.lower().replace("=", '_'))).replace("'", '')}
                VALUES (%s, %s)
                ON CONFLICT (date) DO UPDATE SET
                {symbol.lower().replace("=", '_')} = EXCLUDED.{symbol.lower().replace("=", '_')};
            '''
            
            try:
                cur.execute(insert_query, tuple(volume_data))
                conn.commit()
            except:
                conn.close()
                conn=database_connection()
                cur = conn.cursor()
                cur.execute(insert_query, tuple(volume_data))
                conn.commit()
            
    print(f'{symbol} <-- ADDED')